## 计算子网间的地理距离和网络距离绘制线性关系图
* 同级别联赛，按国家划分子网，先取所有国家*2015赛季*、*第一级联赛*数据(转会前后均为第一级联赛)
* 可视化
* 子网距离计算

In [2]:
import numpy as np
import pandas as pd

### 从_clubs__league.csv_提取含有第一级联赛的记录

In [50]:
df_clubs=pd.read_csv('clubs_league.csv')
df_1lv_2015=df_clubs[(df_clubs['season_id']==2015)&(df_clubs['league_level']=='First tier \xc2\xa0-')]
print 'total records: ',df_1lv_2015.shape[0]
print 'total countries: ',len(df_1lv_2015['country_id'].unique())
print 'total countries(country name): ',len(df_1lv_2015['country_name'].unique())
print 'total clubs: ',len(df_1lv_2015['club_id'].unique())
print 'total clubs(club name): ',len(df_1lv_2015['club_name'].unique())
print 'total leagues: ',len(df_1lv_2015['league_id'].unique())

total records:  1343
total countries:  81
total countries(country name):  81
total clubs:  1161
total clubs(club name):  1160
total leagues:  98


由以上的基础统计信息可以看到
* total clubs < total records: 可能含有同一个俱乐部对应多条记录，需要看看这些重复的俱乐部是什么情况
* total leagues > total countries: 很明显同一个国家的一级联赛可能不止一个，导出这些国家，看看具体是什么原因

In [54]:
# league_group=df_1lv_2015.groupby([df_1lv_2015['country_id'],df_1lv_2015['league_id']]).size()
league_group=df_1lv_2015.groupby([df_1lv_2015['country_name'],df_1lv_2015['country_id'],df_1lv_2015['league_id']]).size().reset_index(name='counts')
# df_league_group=pd.DataFrame(df_1lv_2015.groupby([df_1lv_2015['league_id'],df_1lv_2015['country_id']]).size().rename('cnts'))
country_group=league_group.groupby([league_group['country_name'],league_group['country_id']]).size().reset_index(name='cnts')
# country_group=pd.DataFrame(league_group.groupby([league_group['country_id'],league_group['league_id']]).size().rename('cnts'))
lst_dup_country=country_group['country_name'][country_group['cnts']>1].values.tolist()
# print league_group
# print country_group
print lst_dup_country
# get the records of those countries with multiple first tier league
# df_dup_country=df_1lv_2015[df_1lv_2015['country_id'].isin(lst_dup_country)]
# print df_dup_country

['Chile', 'Colombia', 'Costa Rica', 'Ecuador', 'India', 'Japan', 'Kazakhstan', 'Mexico', 'Peru', 'Romania', 'Uruguay', 'Venezuela']


先逐个分析上述出现同一级别多个联赛的国家是什么原因：
* 分夏、冬季两场锦标赛，俱乐部是同样的俱乐部，但联赛名不同：(Clausura:winter/Apertura:summer)
 美洲：['Chile','Mexico','Uruguay','Ecuador','Peru'(3),]
 亚洲：
 部分国家标记不一样，把目前没展开的锦标赛放到了`currently non-active competitions 部分`
 美洲['Colombia','Venezuela']
* 分多场锦标赛，但是有一场参赛球队数目不同或者是展开类似championship,relegation/playoff之类的比赛
 美洲： ['Costa Rica'(3)]
 亚洲：['Japan'(3),'India'(2)]
 欧洲：['Kazakhstan','Romania'(3)]